In [18]:
import csv
import os
from collections import * 
print("Please make sure you have run filter script")
#file = input('Enter path to Mturk csv')
file = 'Batch_3851433_batch_results.csv'
#comment = input("Enter a reject comment")
comment = 'gg'
rejected_results = list(map(lambda x: x.split('.')[0], os.listdir('../rejected_results')))
approved_results = list(map(lambda x: x.split('.')[0], os.listdir('../results')))

buffer = []
def parse(line):
    line = line.strip().split("\",\"")
    line[0]= line[0][1:] #get rid of first "
    line[-1]= line[-1][:-1] # ... last "
    return line
def cat(line):
    line = "\"" + "\",\"".join(line) + "\"\n"
    return line

with open(file) as csv_file:    
    buffer = csv_file.readlines()
    buffer = list(map(parse, buffer))
    header = buffer[0]
    
    reject = header.index('Reject')
    approve = header.index('Approve') 
    w_id = header.index('WorkerId')
    cmt = header.index('RequesterFeedback')
    
    buffer[0] = cat(header)
    
    for i in range(1,len(buffer)): 
        row = buffer[i]
        while (len(row) <= 30): 
            row.append('')
        if row[w_id] in rejected_results:
            row[reject]='x'
            row[cmt] = comment
        if row[w_id] in approved_results:
            row[approve]='x'
        buffer[i]= cat(row)


with open(file, 'w',newline='') as csv_file:
    csv_file.writelines(buffer)

Please make sure you have run filter script
['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward', 'CreationTime', 'MaxAssignments', 'RequesterAnnotation', 'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds', 'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds', 'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime', 'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime', 'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate', 'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Answer.surveycode', 'Approve', 'Reject']
1
28
2
28
3
28
4
28
5
28
6
28
7
28
8
28
9
28
10
28
11
28
12
28
13
28
14
28
15
28
16
28
17
28
18
28
19
28
20
28
21
28
22
28
23
28
24
28
25
28
26
28
27
28
28
28
29
28
30
28
31
28
32
28
33
28
34
28
35
28
36
28
37
28
38
28
39
28
40
28
41
28
42
28
43
28
44
28
45
28
46
28
47
28
48
28
49
28
50
28
51
28
52
28
53
28
54
28
55
28
56
28
57
28
58
28
59
28


In [50]:
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=','
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])

In [ ]:
import numpy as np
import re
import os
import sys
import subprocess


#Sample filter function
def filter_single_video(video_times, rating_times, video_order, scores,attentions):
    #First check if user watching alines with video length
    #0.5 sec tolerance for black screen at the end
    ret = 0
    bad = 0
    violate = 0
    a = 0
    for k in video_times[:-1]:
        if k < 10000:
            a += 1
    if video_times[-1] < 7000:
        a +=1
    if scores[-1] != max(scores):
        a+=1
    # if scores[3] > scores[2] or scores[1] > scores[0]:
    #    a+=1
    
    if a >0:
        return 1
    return 0 #We don't move this user to rejected folder

#Parse data from user result file 
def parse_results(lines):
    #video_times = list(map(int,lines[2].strip().split(','))) #read times spent on each video  
    #rating_times = list(map(int,lines[3].strip().split(','))) #read times spent on each rating  
    video_times = []; rating_times = []
    video_order = list(map(int,lines[1].strip().split(','))) #read the video order seen by the surveyee
    scores = list(map(int,lines[0].strip().split(','))) #read scores
    attentions = list(map(int,lines[-1].strip().split(','))) #attention checks    
    video_times =  list(map(int,lines[2].strip().split(','))) 
    return video_times, rating_times, video_order, scores, attentions


#input from the cmd line script
result_path = '../results/'
#white_list = '../good/'


move = False
result_files = os.listdir(result_path)
#white_list_files = os.listdir(white_list)

total = 0
approved = 0
reject = 0
pending = 0
for result_file in result_files:
    #filter a single result
    total +=1
    '''
    if result_file in white_list_files:
        approved +=1
        continue
    '''
    result = result_path + result_file
    print(result)
    with open(result, "r") as fp:
        lines = fp.readlines()
        move = False

        #insert customized filter here 
        move = filter_single_video(*parse_results(lines))
        if move == 1:
            reject +=1
            print(result_file)
            os.system("mv {} ../rejected_results/".format(result))
        elif move == 2:
            pending +=1
            os.system("mv {} ../pending/".format(result))
        else:
            approved +=1

print('reject:',reject)
print('approved:',approved)
print('rejection ratio:', reject/(reject+approved))
